# Process Source HEC-RAS Models
Generate FIM Libraries for multiple HEC-RAS Models

In [ ]:
%load_ext autoreload
%autoreload 2

from src import *
from src.setup import *
from src.process import *

### `Parameters`

In [ ]:
collection_name = "" # Define Collection name - example: "ble_12100302_Medina"
# collection_name = "Baxter_0_7_0"

### Instantiate CollectionData, Database, and JobClient objects

In [ ]:
collection = CollectionData(collection_name)
database = Database(collection)
jobclient = JobClient(collection)

### Get Working Models

In [ ]:
# Obtain models data from what is currently on disk.
model_ids = collection.get_models()

### Run `conflate_model`

In [ ]:
conflate_step_processor = ConflateModelStepProcessor(collection, model_ids)
conflate_step_processor.execute_step(jobclient, database, timeout=5)

### Load Conflation Information to Database

In [ ]:
succeeded_and_unknown_status_models = [
    job_status[0]
    for job_status in conflate_step_processor.job_statuses["succeeded"]
    + conflate_step_processor.job_statuses["unknown"]
]
load_conflation(succeeded_and_unknown_status_models, database)

### Update `network_to_id` Table in Database
Build a modified network by skipping over reaches eclipsed by adjacent reaches

In [ ]:
update_network(database)

### Get Working Reaches

In [ ]:
reaches = database.get_reaches_by_models(model_ids)
reach_data = [(data[0], data[2]) for data in reaches]

### Run `extract_submodel` step

In [ ]:
submodel_step_processor = GenericReachStepProcessor(collection, reach_data, "extract_submodel")
submodel_step_processor.execute_step(jobclient, database, timeout=5)

### Run `create_ras_terrain` step

In [ ]:
valid_reaches = [
    (job_status[0], "")
    for job_status in submodel_step_processor.job_statuses["succeeded"]
    + submodel_step_processor.job_statuses["unknown"]
]
terrain_step_processor = GenericReachStepProcessor(collection, valid_reaches, "create_ras_terrain")
terrain_step_processor.execute_step(jobclient, database, timeout=3)

### Run `create_model_run_normal_depth` step

In [ ]:
valid_reaches = [
    (job_status[0], "")
    for job_status in terrain_step_processor.job_statuses["succeeded"]
    + terrain_step_processor.job_statuses["unknown"]
]
create_model_step_processor = GenericReachStepProcessor(collection, valid_reaches, "create_model_run_normal_depth")
create_model_step_processor.execute_step(jobclient, database, timeout=10)

### Run `run_incremental_normal_depth` step

In [ ]:
valid_reaches = [
    (job_status[0], "")
    for job_status in create_model_step_processor.job_statuses["succeeded"]
    + create_model_step_processor.job_statuses["unknown"]
]
nd_step_processor = GenericReachStepProcessor(collection, reach_data, "run_incremental_normal_depth")
nd_step_processor.execute_step(jobclient, database, timeout=25)

### Initial `run_known_wse` and Initial `create_rating_curves_db`

In [ ]:
outlet_reaches = [data[0] for data in reaches if data[1] is None]
execute_ikwse_for_network(
    [(reach, None) for reach in outlet_reaches],
    collection,
    database,
    jobclient,
    timeout=20,
)

### Final `run_known_wse`

In [ ]:
valid_reaches = [
    (data[0], data[1])
    for data in reaches
    if data[1] is not None
    and data[0]
    in [
        job_status[0]
        for job_status in nd_step_processor.job_statuses["succeeded"] + nd_step_processor.job_statuses["unknown"]
        # to do: also check if ind successful for downstream reach, and that ikwse was successful for this reach
    ]
]

kwse_step_processor = KWSEStepProcessor(collection, valid_reaches)
kwse_step_processor.execute_step(jobclient, database, timeout=180)

### Final `create_rating_curves_db` step

In [ ]:
valid_reaches = [
    job_status[0]
    for job_status in kwse_step_processor.job_statuses["succeeded"] + kwse_step_processor.job_statuses["unknown"]
]
rc_step_processor = GenericReachStepProcessor(collection, reach_data, "create_rating_curves_db")
rc_step_processor.execute_step(jobclient, database, timeout=15)

### Merge Rating Curves

In [ ]:
load_all_rating_curves(database)

### Run `create_fim_lib` Step

In [ ]:
valid_reaches = [
    job_status[0]
    for job_status in rc_step_processor.job_statuses["succeeded"] + rc_step_processor.job_statuses["unknown"]
]
fimlib_step_processor = GenericReachStepProcessor(collection, reach_data, "create_fim_lib")
fimlib_step_processor.execute_step(jobclient, database, timeout=150)

### Create extent library step

In [ ]:
create_extent_lib(collection)

### Create Flows2FIM Start Reaches File

In [ ]:
outlet_reaches = [data[0] for data in reaches if data[1] is None]
create_f2f_start_file(outlet_reaches, collection.f2f_start_file)